# Importing

Import the necessary libraries

In [ ]:
import os
from os.path import join

import matplotlib.pyplot as plt
import numpy as np
import PIL.ImageOps
from PIL import Image

import IshiharaCreateSimDal as IshiDalton
import IshiharaSimulate.plates as plates
import IshiharaSimulate.utils as utils
from IshiharaMC import ishiharaMC
from PNGToSVG import pngtosvg

# Processing images

Loading, converting to monochrome and then saving as SVG files

In [ ]:
old_file = os.getcwd() + os.sep + join("Tutorial", "TestData", "61.jpg")
new_file = os.getcwd() + os.sep + join("Tutorial", "Output", "61.svg")

# Loading,converting to monochrome and then saving as SVG files
image = Image.open(old_file)
image = PIL.ImageOps.invert(image)
image = image.convert("1").convert("RGBA")
svg_image = pngtosvg.rgba_image_to_svg_contiguous(image)
with open(new_file, "w") as text_file:
    text_file.write(svg_image)

### Plot an example MNIST image

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(image)

### Processing the paths and visualizing them

In [ ]:
# Reading the file and considering only paths with mean RGB smaller than 50 (Black)
path_strings, path_style = utils.get_d_style(new_file)
paths = []
black_index = []
for p, s in zip(path_strings, path_style):
    strip = utils.clean_strip(p)
    paths += [strip]
    if utils.get_mean_RGB(s) < 50:
        black_index += [len(paths) - 1]

# Normalizing the locations using the bounding box
paths = utils.normaize_paths(paths)
# Choosing only black paths
paths = [paths[i] for i in black_index]
# paths_org keeps the path hierarchy
paths_org = paths
# paths contains all paths unnested
paths = [x1 for x in paths_org for x1 in x]

In [ ]:
# plotting the paths in a simple way
cols = ["g", "r", "b"]
plt.figure(figsize=(4, 4))
# plt.subplots()
for i, path in enumerate(paths):
    xs, ys = zip(*path)  # create lists of x and y values
    plt.plot(xs, ys, color=cols[i], label="path " + str(i + 1))

plt.legend(loc="lower right")
plt.show()

### Perfrom MC simulation with different radiuses

Note: this may take a while

In [ ]:
output = np.array(
    ishiharaMC.createPlate(paths, num=[4000, 2 * 4000, 4 * 4000, 8 * 4000])
)

In [ ]:
idx = [[ishiharaMC.circinPoly(x1, output) for x1 in x] for x in paths_org]
idx_new = []
for i, arr in enumerate(idx):
    if len(arr) == 1:
        idx_new += [arr[0]]
    else:
        outer = utils.get_outer_path(paths_org[i])

        temp = np.array([True] * arr[0].shape[0])
        for x in arr[:outer] + arr[outer + 1 :]:
            temp_or = np.logical_or(arr[outer], x)
            temp_and = np.logical_and(arr[outer], x)
            temp = np.logical_and(
                temp, np.logical_and(temp_or, np.logical_not(temp_and))
            )
        idx_new += [temp]

idx_final = idx_new[0]
for x in idx_new[1:]:
    idx_final = np.logical_or(idx_final, x)

### Plot Ishihara-style MNIST images

Plot Ishihara-style MNIST images for the figure 8 with different colors and plates

In [ ]:
plt.subplot(131)
ishiharaMC.plotIshi(output[np.logical_not(idx_final)], color="w")
ishiharaMC.plotIshi(output[idx_final], color="k")

plt.subplot(132)
ishiharaMC.plotIshi(output[np.logical_not(idx_final)], color="k")
ishiharaMC.plotIshi(output[idx_final], color="w")

plt.subplot(133)
ishiharaMC.plotIshi(output[np.logical_not(idx_final)], color="g")
ishiharaMC.plotIshi(output[idx_final], color="r")
plt.show()

### Plot Ishihara-style MNIST image (for Plate 1)

In [ ]:
plates.plot_plate_generic(output, idx_final, plates.Plates_Dict[1])
plt.show()

### Plot Ishihara-style MNIST images with all Ishihara plates + the baseline plates

Plot Ishihara-style MNIST images for the figure 8 with all Ishihara plates + the baseline plates: only dots, only circles and random colors plate

In [ ]:
Plate_names = [
    "(a) Only dots",
    "(b) Only circles",
    "(c) Plate 1",
    "(d) Plate 2",
    "(e) Plate 3",
    "(f) Plate 4",
    "(g) Plate 5",
    "(h) Plate 6",
    "(i) Plate 7",
    "(j) Plate 8",
    "(k) Plate 9",
    "(l) Plate 10",
    "(m) Plate 11",
    "(n) Plate 12",
    "(o) Plate 13",
    "(p) Plate 16",
    "(q) Plate 17",
    "(r) Random colors",
]

plate_functions = [
    plates.create_dots_img,
    plates.create_circles_img,
    plates.plot_plate_generic,
    plates.plot_plate2,
    plates.plot_plate3,
    plates.plot_plate4,
    plates.plot_plate5,
    plates.plot_plate6,
    plates.plot_plate7,
    plates.plot_plate_generic,
    plates.plot_plate_generic,
    plates.plot_plate_generic,
    plates.plot_plate_generic,
    plates.plot_plate_generic,
    plates.plot_plate_generic,
    plates.plot_plate16_17,
    plates.plot_plate16_17,
]

Plates_Dict_keys = [21, 20, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17]
plt.subplots(figsize=(21, 42))
for j, plate in enumerate(Plates_Dict_keys):
    plt.subplot(6, 3, j + 1)
    plate_functions[j](output, idx_final, plates.Plates_Dict[plate])
    plt.xlabel(Plate_names[j], fontsize=24, labelpad=10)

# Plotting the random colors
plt.subplot(6, 3, 18)
plt.xlabel(Plate_names[j + 1], fontsize=24, labelpad=10)

plates.plot_plate_random_color(output, idx_final)

### Ishihara-style MNIST images with manipulations 

In [ ]:
upper_half = lambda x: x[1] > 0
right_half = lambda x: x[0] > 0
above_line = lambda x: x[0] - x[1] < 0
random15 = lambda x: np.random.random_sample() > 0.85
XOR = lambda x: (x[0] > 0) ^ (x[1] > 0)
chess_board = lambda x: (x[0] * 2 % 2 > 1) ^ (x[1] * 2 % 2 > 1)

Manipulation_Functions = [
    upper_half,
    right_half,
    above_line,
    random15,
    XOR,
    chess_board,
]

In [ ]:
Plate_names = [
    "(a) Only dots",
    "(b) Only circles",
    "(c) Plate 1",
    "(d) Plate 2",
    "(e) Plate 4",
    "(f) Plate 8",
    "(g) Plate 10",
    "(h) Plate 16",
    "(i) Random colors",
]

plate_functions = [
    plates.create_dots_img,
    plates.create_circles_img,
    plates.plot_plate_generic,
    plates.plot_plate2,
    plates.plot_plate4,
    plates.plot_plate_generic,
    plates.plot_plate_generic,
    plates.plot_plate16_17,
]

Plates_Dict_keys = [21, 20, 1, 2, 4, 8, 10, 16]
plt.subplots(figsize=(21, 28))
for j, plate in enumerate(Plates_Dict_keys):
    plt.subplot(4, 3, j + 1)
    # plt.axis('off')
    plate_functions[j](output, idx_final, plates.Plates_Dict[plate])
    # plt.title(Plate_names[j])
    plt.xlabel(Plate_names[j], fontsize=24, labelpad=10)

# Plotting the random colors
plt.subplot(4, 3, 9)
plt.xlabel(Plate_names[j + 1], fontsize=24, labelpad=10)
plates.plot_plate_random_color(output, idx_final)

##########################
plt.subplot(4, 3, 10)
above_line = lambda x: x[0] - x[1] < 0
plates.plot_transform(
    output, idx_final, above_line, plates.plot_plate_generic, plates.Plates_Dict[1]
)
plt.xlabel("(j) Flip above identity, plate 1", fontsize=24, labelpad=10)

##########################
plt.subplot(4, 3, 11)
chess_board = lambda x: (x[0] > 0) ^ (x[1] > 0)
plates.plot_transform(
    output, idx_final, chess_board, plates.plot_plate_generic, plates.Plates_Dict[4]
)
plt.xlabel("(k) Flip XOR, plate 4", fontsize=24, labelpad=10)

##########################
# Sim 8 p
types = {
    1: None,
    2: "Sim_d",
    3: "Sim_p",
    4: "Sim_t",
    5: "Dal_d",
    6: "Dal_p",
    7: "Dal_t",
}
color_blindness = types[3][-1]
new_plate = color_blindness
folder = join(os.getcwd(), "output_dir")
fname = os.getcwd() + os.sep + join("output_dir", "Plate_8_noaxis.png")

gamma = 2.4
plt.subplot(4, 3, 12)
image = IshiDalton.create_sim_colorblind_file(
    fname, folder, new_plate, color_blindness, gamma, True
)
plt.imshow(image)
plt.xlabel("(l) Simulate protanopia, plate 8", fontsize=24, labelpad=10)
##########################

plt.show()

### Daltonize the figures and simulate color blindness plates

In [ ]:
Plate_names = [
    "(a) Horizontal, plate 1",
    "(b) Vertical, plate 1",
    "(c) Identity function, plate 1",
    "(d) Random 15%, plate 1",
    "(e) XOR, plate 1",
    "(f) Chessboard, plate 1",
]
plt.subplots(figsize=(21, 28))

for j, man in enumerate(Manipulation_Functions):
    plt.subplot(4, 3, j + 1)
    plates.plot_transform(
        output, idx_final, man, plates.plot_plate_generic, plates.Plates_Dict[1]
    )
    plt.xlabel(Plate_names[j], fontsize=24, labelpad=10)
    ##########################
type_names = ["protanopia", "deuteranopia", "tritanopia"]

folder = join(os.getcwd(), "output_dir")
fname = os.getcwd() + os.sep + join("output_dir", "Plate_8_noaxis.png")

gamma = 2.4
types_short = ["d", "p", "t"]
prefix = ["(g) ", "(h) ", "(i) "]

for j, t in enumerate(types_short):
    color_blindness = t
    new_plate = color_blindness
    plt.subplot(4, 3, 6 + j + 1)
    image = IshiDalton.create_sim_colorblind_file(
        fname, folder, new_plate, color_blindness, gamma, True
    )
    plt.imshow(image)
    plt.xlabel(
        prefix[j] + "Simulate " + type_names[j] + ", plate 8", fontsize=24, labelpad=10
    )
    prefix = ["(j) ", "(k) ", "(l) "]

for j, t in enumerate(types_short):
    color_blindness = t
    new_plate = color_blindness
    plt.subplot(4, 3, 9 + j + 1)
    image = IshiDalton.create_daltonize_colorblind_file(
        fname, folder, new_plate, color_blindness, gamma, True
    )
    plt.imshow(image)
    plt.xlabel(
        prefix[j] + "Daltonize " + type_names[j] + ", plate 8", fontsize=24, labelpad=10
    )

### Example for generating plate with XOR transformation and Protanopia simulation plate

In [ ]:
plt.subplots(figsize=(7, 7))
plates.plot_transform(
    output, idx_final, XOR, plates.plot_plate_generic, plates.Plates_Dict[1]
)
plt.xlabel("XOR_plate_1", fontsize=24, labelpad=10)
plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)

##########################
type_names = ["protanopia", "deuteranopia", "tritanopia"]
folder = join(os.getcwd(), "output_dir")
fname = os.getcwd() + os.sep + join("output_dir", "Plate_8_noaxis.png")
gamma = 2.4
color_blindness = "p"
new_plate = color_blindness
plt.subplots(figsize=(7, 7))
image = IshiDalton.create_sim_colorblind_file(
    fname, folder, new_plate, color_blindness, gamma, True
)
plt.imshow(image)
plt.xlabel("protanopia_plate_1", fontsize=24, labelpad=10)
plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)

In [ ]:
#os.system('jupyter nbconvert --to html TransformMNIST.html')